# Imports

In [ ]:
import torch
from torch.amp import autocast, GradScaler
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as T
import torchvision.transforms.functional as F
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign
import os
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image, ImageDraw
import contextlib
import io
import time
from datetime import datetime
from tqdm import tqdm
from torchvision.transforms import ColorJitter, GaussianBlur
import random
import matplotlib.patches as patches


In [ ]:
class HazmatDataset(Dataset):
    def __init__(self, data_dir, annotations_file, transforms=None):
        self.data_dir = data_dir
        self.transforms = transforms

        # Load annotations
        with open(annotations_file) as f:
            data = json.load(f)

        self.images = {img['id']: img for img in data['images']}
        self.annotations = data['annotations']

        # Create image_id to annotations mapping
        self.img_to_anns = {}
        for ann in self.annotations:
            img_id = ann['image_id']
            if img_id not in self.img_to_anns:
                self.img_to_anns[img_id] = []
            self.img_to_anns[img_id].append(ann)

        self.ids = list(self.images.keys())

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.images[img_id]

        # Load image
        img_path = os.path.join(self.data_dir, 'images', img_info['file_name'])
        img = Image.open(img_path).convert('RGB')

        # Get annotations
        anns = self.img_to_anns.get(img_id, [])

        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for ann in anns:
            bbox = ann['bbox']
            # Convert [x, y, w, h] to [x1, y1, x2, y2]
            x_min = bbox[0]
            y_min = bbox[1]
            x_max = bbox[0] + bbox[2]
            y_max = bbox[1] + bbox[3]

            # Filter out degenerate boxes
            if x_max > x_min and y_max > y_min:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(ann['category_id'])
                areas.append(ann['area'])
                iscrowd.append(ann['iscrowd'])

        # Convert to tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id]),
            'area': areas,
            'iscrowd': iscrowd
        }

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.ids)


# Functions

In [ ]:
class ToTensor(object):
    def __call__(self, image, target):
        # Convert PIL image to tensor
        image = F.to_tensor(image)
        return image, target



def collate_fn(batch):
    return tuple(zip(*batch))

def train_one_epoch(model, optimizer, data_loader, device, scaler):
    model.train()
    total_loss = 0
    total_classifier_loss = 0
    total_box_reg_loss = 0
    total_objectness_loss = 0
    total_rpn_box_reg_loss = 0

    # Voeg tqdm toe om de voortgang te tonen
    progress_bar = tqdm(data_loader, desc="Training", leave=True)
    
    for images, targets in progress_bar:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Wrap the forward pass in autocast
        with autocast(device_type='cuda'):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        # Scale the loss and call backward
        scaler.scale(losses).backward()
        # Unscales the gradients and calls or skips optimizer.step()
        scaler.step(optimizer)
        # Updates the scale for next iteration
        scaler.update()

        # Bereken de totalen
        total_loss += losses.item()
        total_classifier_loss += loss_dict['loss_classifier'].item()
        total_box_reg_loss += loss_dict['loss_box_reg'].item()
        total_objectness_loss += loss_dict['loss_objectness'].item()
        total_rpn_box_reg_loss += loss_dict['loss_rpn_box_reg'].item()

        # Update tqdm-balk
        progress_bar.set_postfix({
            "Loss": f"{losses.item():.4f}",
            "Classifier": f"{loss_dict['loss_classifier'].item():.4f}",
            "BoxReg": f"{loss_dict['loss_box_reg'].item():.4f}",
        })

    avg_loss = total_loss / len(data_loader)
    avg_classifier_loss = total_classifier_loss / len(data_loader)
    avg_box_reg_loss = total_box_reg_loss / len(data_loader)
    avg_objectness_loss = total_objectness_loss / len(data_loader)
    avg_rpn_box_reg_loss = total_rpn_box_reg_loss / len(data_loader)

    return avg_loss, avg_classifier_loss, avg_box_reg_loss, avg_objectness_loss, avg_rpn_box_reg_loss



# Load ground truth annotations
coco_val = COCO('data/data_faster_rcnn/val/annotations/instances_val.json')

# Prepare predictions in COCO format
# Assuming you have a function to convert model outputs to COCO format
# Conversion to COCO Format
def convert_to_coco_format(outputs, image_ids):
    coco_results = []
    for output, image_id in zip(outputs, image_ids):
        boxes = output['boxes'].cpu().numpy()
        scores = output['scores'].cpu().numpy()
        labels = output['labels'].cpu().numpy()
        
        for box, score, label in zip(boxes, scores, labels):
            coco_results.append({
                'image_id': image_id,
                'category_id': int(label),
                'bbox': [box[0], box[1], box[2] - box[0], box[3] - box[1]],
                'score': float(score)
            })
    return coco_results

# Validation Function
def validate(model, data_loader, coco_gt, device):
    model.eval()
    results = []

    # Add tqdm
    progress_bar = tqdm(data_loader, desc="Validation", leave=True)

    with torch.no_grad():
        for images, targets in progress_bar:
            images = list(image.to(device) for image in images)
            outputs = model(images)
            
            image_ids = [target['image_id'].item() for target in targets]
            coco_results = convert_to_coco_format(outputs, image_ids)
            results.extend(coco_results)

            # Update tqdm-bar
            progress_bar.set_postfix({"Processed": len(results)})

    if not results:
        print("No predictions generated. Skipping evaluation.")
        return [0.0] * 6  # Return dummy metrics for empty results

    # Suppress COCOeval output
    with contextlib.redirect_stdout(io.StringIO()):
        coco_dt = coco_gt.loadRes(results)
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

    return coco_eval.stats


# Custom backbone to return a dictionary of feature maps
class BackboneWithChannels(torch.nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        x = self.backbone(x)
        return {'0': x}
    
# Function to create a subset of the dataset
def create_subset(dataset, percentage):
    """
    Create a subset of the dataset based on the given percentage.
    
    Parameters:
    - dataset: The full dataset.
    - percentage: The fraction of the dataset to use (value between 0.0 and 1.0).
    
    Returns:
    - subset: A subset of the dataset containing the specified percentage of data.
    """
    if not (0.0 < percentage <= 1.0):
        raise ValueError("Percentage must be between 0.0 and 1.0.")
    
    # Determine the subset size
    total_samples = len(dataset)
    subset_size = int(total_samples * percentage)
    
    # Shuffle and select a random subset of indices
    indices = list(range(total_samples))
    random.shuffle(indices)
    subset_indices = indices[:subset_size]
    
    return Subset(dataset, subset_indices)

def create_directory(base_path="data/models"):
    """
    Create a directory inside the base path named 'faster-rcnn-finetuned-{date}' 
    to store models and logs. The name includes the current date and time in the format 'DD-MM-YYYY HH:MM:SS'.

    Parameters:
    - base_path (str): Base directory where the new directory will be created.

    Returns:
    - directory_path (str): Full path to the created directory.
    """
    # Get the current date and time
    current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    
    # Define the full directory path
    directory_name = f"faster-rcnn-finetuned-{current_time}"
    directory_path = os.path.join(base_path, directory_name)
    
    # Create the directory
    os.makedirs(directory_path, exist_ok=True)
    
    print(f"Directory created: {directory_path}")
    return directory_path

def train_model(directory, model, optimizer, train_loader, device, train_metrics_list, best_val_map, lr_scheduler, val_loader, coco_val, scaler, epoch):
    
    epoch+=1
    # Start the timer
    start_time = time.time()
    
    # Train for one epoch
    train_loss, train_classifier_loss, train_box_reg_loss, train_objectness_loss, train_rpn_box_reg_loss = train_one_epoch(
        model, optimizer, train_loader, device, scaler)
    
    # Validate and get all COCO-metrics
    val_metrics = validate(model, val_loader, coco_val, device)
    val_map = val_metrics[0]  # mAP@IoU=0.50:0.95
    
    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    
    # Obtain the current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    
    # Prepare data for logging
    data = {
        "epoch": epoch,
        "time_elapsed": (int(minutes), int(seconds)),
        "learning_rate": current_lr,
        "train_loss": train_loss,
        "classifier_loss": train_classifier_loss,
        "box_reg_loss": train_box_reg_loss,
        "objectness_loss": train_objectness_loss,
        "rpn_box_reg_loss": train_rpn_box_reg_loss,
        "val_metrics": val_metrics
    }
    
    # Append current epoch data to metrics list
    train_metrics_list.append(data)
    
    # Print summary for this epoch
    print(f"📊 Epoch {epoch} | ⏳ Time: {int(minutes)}m {int(seconds)}s | 🔄 LR: {current_lr:.6f}")
    print(f"📉 Train Loss: {train_loss:.4f} | 🎯 Classifier: {train_classifier_loss:.4f} | 📦 Box Reg: {train_box_reg_loss:.4f}")
    print(f"🔍 Objectness: {train_objectness_loss:.4f} | 🗂️ RPN Box Reg: {train_rpn_box_reg_loss:.4f}")
    print(f"🧪 mAP | 🟢 mAP@IoU=0.50:0.95: {val_metrics[0]:.4f} | 🔵 mAP@IoU=0.50: {val_metrics[1]:.4f} | 🟣 mAP@IoU=0.75: {val_metrics[2]:.4f}")
    print(f"📏 Small mAP: {val_metrics[3]:.4f} | 📐 Medium mAP: {val_metrics[4]:.4f} | 📏 Large mAP: {val_metrics[5]:.4f}")
    
    # Save epoch data to a log file
    save_epoch_data(directory, data)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Save the latest checkpoint with all metrics
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_map': val_map,
        'train_metrics_list': train_metrics_list  # Save all metrics
    }
    torch.save(checkpoint, os.path.join(directory, "latest_model.pth"))
    
    # Save the best model if the val_map is the highest so far
    if val_map > best_val_map:
        best_val_map = val_map
        torch.save(checkpoint, os.path.join(directory, "best_model.pth"))
    
    return best_val_map
        


def save_epoch_data(directory, data):
    """
    Save training statistics for each epoch in a text file.

    Parameters:
    - directory (str): Path to the directory.
    - data (dict): Contains data on metrics such as epoch, losses, and validation metrics.
    """
    log_file_path = os.path.join(directory, "training_log.txt")
    
    lines = [
        f"Epoch {data['epoch']} | Time: {data['time_elapsed'][0]}m {data['time_elapsed'][1]}s | LR: {data['learning_rate']:.10f}\n",
        f"Train Loss: {data['train_loss']:.4f} | Classifier: {data['classifier_loss']:.4f} | Box Reg: {data['box_reg_loss']:.4f}\n",
        f"Objectness: {data['objectness_loss']:.4f} | RPN Box Reg: {data['rpn_box_reg_loss']:.4f}\n",
        f"Validation Metrics: | mAP@IoU=0.50:0.95: {data['val_metrics'][0]:.4f} | mAP@IoU=0.50: {data['val_metrics'][1]:.4f} | mAP@IoU=0.75: {data['val_metrics'][2]:.4f}\n",
        f"Small mAP: {data['val_metrics'][3]:.4f} | Medium mAP: {data['val_metrics'][4]:.4f} | Large mAP: {data['val_metrics'][5]:.4f}\n\n"
    ]
    with open(log_file_path, "a") as log_file:
        log_file.writelines(lines)


# Functions Data Augmentation

In [ ]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        # Check image type
        if not isinstance(image, (torch.Tensor, Image.Image)):
            raise TypeError(f"Unsupported image type: {type(image)}. Expected torch.Tensor or PIL.Image.")
        
        if torch.rand(1) < self.prob:
            if isinstance(image, torch.Tensor):
                width = image.shape[-1]
                image = F.hflip(image)
            else:
                width, _ = image.size
                image = F.hflip(image)
            
            # Flip bounding boxes
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]  # Flip x-coordinates
            target["boxes"] = bbox
        return image, target

class RandomBrightnessCont(object):
    def __init__(self, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5):
        self.color_jitter = ColorJitter(brightness, contrast, saturation, hue)
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p:
            image = self.color_jitter(image)
        return image, target

class RandomBlur(object):
    def __init__(self, kernel_size=3, p=0.5):
        self.blur = GaussianBlur(kernel_size)
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p:
            image = self.blur(image)
        return image, target

class RandomRotate(object):
    def __init__(self, angle_range=10, p=0.5):
        self.angle_range = angle_range
        self.p = p

    def __call__(self, image, target):
        # Check image type
        if not isinstance(image, (torch.Tensor, Image.Image)):
            raise TypeError(f"Unsupported image type: {type(image)}. Expected torch.Tensor or PIL.Image.")

        if random.random() < self.p:
            angle = random.uniform(-self.angle_range, self.angle_range)
            if isinstance(image, torch.Tensor):
                image = F.to_pil_image(image)
                image = F.rotate(image, angle)
                image = F.to_tensor(image)
            else:
                image = F.rotate(image, angle)

            # Rotate bounding boxes (as in your code)
            boxes = target['boxes']
            if len(boxes) > 0:
                # Rotate logic here
                pass  # (Keep your rotation logic)

        return image, target

    
class RandomZoom(object):
    def __init__(self, zoom_range=(1.0, 2.0), p=0.5):
        self.zoom_range = zoom_range
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p:
            boxes = target['boxes']
            if len(boxes) == 0:
                return image, target

            box_idx = random.randint(0, len(boxes) - 1)
            x1, y1, x2, y2 = boxes[box_idx].numpy()

            width, height = image.size
            center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
            box_width, box_height = x2 - x1, y2 - y1

            zoom_factor = random.uniform(*self.zoom_range)
            crop_width = box_width / zoom_factor
            crop_height = box_height / zoom_factor

            crop_x1 = max(0, center_x - crop_width / 2)
            crop_y1 = max(0, center_y - crop_height / 2)
            crop_x2 = min(width, center_x + crop_width / 2)
            crop_y2 = min(height, center_y + crop_height / 2)

            image = image.crop((int(crop_x1), int(crop_y1), int(crop_x2), int(crop_y2)))
            target['boxes'][:, [0, 2]] -= crop_x1
            target['boxes'][:, [1, 3]] -= crop_y1
            target['boxes'][:, [0, 2]] = target['boxes'][:, [0, 2]].clamp(0, crop_x2 - crop_x1)
            target['boxes'][:, [1, 3]] = target['boxes'][:, [1, 3]].clamp(0, crop_y2 - crop_y1)

        return image, target

    
def get_augmented_transform(train):
    """
    Get transform pipeline with augmentations for training or validation
    """
    transforms = []
    
    if train:
        # Applies a series of data augmentations specifically for the training set
        transforms.extend([
            RandomHorizontalFlip(0.5),  # Horizontally flips the image with a 50% probability
            RandomBrightnessCont(  # Adjusts brightness, contrast, saturation, and hue with specified ranges
                brightness=0.3, 
                contrast=0.4, 
                saturation=0.5, 
                hue=0.5, 
                p=0.5  # Applies these adjustments with a 50% probability
            ),
            RandomBlur(kernel_size=3, p=0.3),  # Applies Gaussian blur with a kernel size of 3, 30% chance
            # RandomRotate(angle_range=10, p=0.3),  # Rotates the image by -10 to +10 degrees, 30% chance
            RandomZoom(zoom_range=(0.05, 0.9), p=0.6)  # Zooms the image by a factor between 0.05 and 0.9, 60% chance
        ])

    # Converts the image to a tensor for model input
    transforms.append(ToTensor())
    
    return Compose(transforms)

def visualize_augmentations(dataset, num_samples=5):
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 4 * num_samples))
    
    for i in range(num_samples):
        idx = random.randint(0, len(dataset) - 1)
        orig_img, orig_target = dataset[idx]
        
        if isinstance(orig_img, torch.Tensor):
            orig_img_np = orig_img.permute(1, 2, 0).numpy()
        else:
            orig_img_np = np.array(orig_img)
        
        axes[i, 0].imshow(orig_img_np)
        axes[i, 0].set_title('Original')
        
        aug_img, aug_target = dataset[idx]
        aug_img_np = aug_img.permute(1, 2, 0).numpy() if isinstance(aug_img, torch.Tensor) else np.array(aug_img)
        
        axes[i, 1].imshow(aug_img_np)
        axes[i, 1].set_title('Augmented')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Define your augmentations
augmentations = get_augmented_transform(train=True)

# Create directories to save augmented data
augmented_images_dir = 'data/data_faster_rcnn/train_aug/images'
os.makedirs(augmented_images_dir, exist_ok=True)

# Remove all files in the augmented_images_dir
if os.path.exists(augmented_images_dir):
    for file in os.listdir(augmented_images_dir):
        file_path = os.path.join(augmented_images_dir, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove the file or symlink
            elif os.path.isdir(file_path):
                os.rmdir(file_path)  # Remove the directory (if empty)
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")
else:
    print(f"Directory {augmented_images_dir} does not exist.")

# Define the path for augmented annotations
augmented_annotations_file = 'data/data_faster_rcnn/train_aug/annotations/instances_aug.json'

# Function to save dictionaries as JSON files
def save_json(data, json_path):
    with open(json_path, "w") as file:
        json.dump(data, file)

# Function to convert tensors to lists for JSON serialization
def tensor_to_list(obj):
    if isinstance(obj, torch.Tensor):
        return obj.tolist()  # Convert Tensor to list
    elif isinstance(obj, dict):
        return {key: tensor_to_list(value) for key, value in obj.items()}  # Recursively convert dicts
    elif isinstance(obj, list):
        return [tensor_to_list(item) for item in obj]  # Recursively convert lists
    else:
        return obj  # Return other types as is

# Check if the augmented annotations file exists, if not create it
if os.path.exists(augmented_annotations_file):
    os.remove(augmented_annotations_file)  # Delete the existing file
    print(f"Deleted existing annotations file: {augmented_annotations_file}")
    
    
# Create a default structure for the augmented annotations
categories_list = [
    {"id": 1, "name": "hazmat code"}
]

augmented_annotations = {
    'images': [],
    'annotations': [],
    'categories': categories_list
}
save_json(augmented_annotations, augmented_annotations_file)
print(f"Created new annotations file: {augmented_annotations_file}")


# Counter for image and annotation IDs
img_id = len(augmented_annotations['images'])
ann_id = len(augmented_annotations['annotations'])

# Number of augmented images to create
num_images_to_generate = 1000  # Set this to your desired limit

# Iterate through the dataset with progress tracking
generated_count = 0  # Counter for how many images we have generated
for idx in tqdm(range(len(original_dataset)), desc="Augmenting dataset"):
    if generated_count >= num_images_to_generate:
        break  # Stop when the desired number of augmented images is created
    
    img, target = original_dataset[idx]
    
    # Apply augmentations
    aug_img, aug_target = augmentations(img, target)
    
    # Save augmented image
    aug_img_path = os.path.join(augmented_images_dir, f'aug_{img_id}.jpg')
    if isinstance(aug_img, torch.Tensor):
        aug_img = F.to_pil_image(aug_img)
    aug_img.save(aug_img_path)
    
    # Update image annotation
    augmented_annotations['images'].append({
        'id': img_id,
        'file_name': os.path.basename(aug_img_path),
        'width': aug_img.width,
        'height': aug_img.height
    })
    
    # Update annotations
    for box, label in zip(aug_target['boxes'], aug_target['labels']):
        augmented_annotations['annotations'].append({
            'id': ann_id,
            'image_id': img_id,
            'category_id': label.item(),
            'bbox': [
                box[0].item(), box[1].item(), 
                box[2].item() - box[0].item(), 
                box[3].item() - box[1].item()
            ],
            'area': (box[2] - box[0]) * (box[3] - box[1]),
            'iscrowd': 0
        })
        ann_id += 1

    # Update the image and annotation IDs
    img_id += 1
    generated_count += 1  # Increase the generated image counter

# Convert tensors to lists before saving the annotations
augmented_annotations = tensor_to_list(augmented_annotations)

# Save augmented annotations
save_json(augmented_annotations, augmented_annotations_file)

print(f"Generated {generated_count} augmented images.")


In [ ]:
# Visualization Step
def visualize_augmented_data(annotations_file, images_dir, num_to_display=5):
    """
    Visualizes augmented images with bounding boxes.
    """
    # Load annotations
    with open(annotations_file, 'r') as f:
        augmented_annotations = json.load(f)
    
    # Get image and annotation mapping
    images = {img['id']: img for img in augmented_annotations['images']}
    annotations = augmented_annotations['annotations']
    
    # Group annotations by image_id
    annotations_by_image = {}
    for ann in annotations:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        annotations_by_image[img_id].append(ann)
    
    # Visualize specified number of images
    for img_id, img_info in list(images.items())[:num_to_display]:
        img_path = os.path.join(images_dir, img_info['file_name'])
        img = Image.open(img_path)
        
        # Plot image
        fig, ax = plt.subplots(1, figsize=(10, 8))
        ax.imshow(img)
        ax.set_title(f"Image ID: {img_id}")
        
        # Plot bounding boxes
        for ann in annotations_by_image.get(img_id, []):
            bbox = ann['bbox']
            rect = patches.Rectangle(
                (bbox[0], bbox[1]), bbox[2], bbox[3], 
                linewidth=2, edgecolor='red', facecolor='none'
            )
            ax.add_patch(rect)
            ax.text(
                bbox[0], bbox[1] - 5, f"Hazmat Code", 
                color='red', fontsize=12, bbox=dict(facecolor='white', alpha=0.8)
            )
        
        plt.axis('off')
        plt.show()

# Call the visualization function
visualize_augmented_data(
    annotations_file=augmented_annotations_file,
    images_dir=augmented_images_dir,
    num_to_display=200  # Adjust this number to control how many images to display
)


In [ ]:
data_mock = {
    "epoch": 5,
    "time_elapsed": [12, 34],  # 12 minutes, 34 seconds
    "learning_rate": 0.000123,
    "train_loss": 0.5678,
    "classifier_loss": 0.1234,
    "box_reg_loss": 0.2345,
    "objectness_loss": 0.3456,
    "rpn_box_reg_loss": 0.4567,
    "val_metrics": [0.6543, 0.8765, 0.7654, 0.1234, 0.2345, 0.3456]  # Mocked mAP values
}

save_epoch_data("./data", data_mock)


In [ ]:
!nvidia-smi

In [ ]:
!kill -9 36090

# Model initialisation

In [ ]:
device = torch.device('cuda:0')
print(f"Loading model on {device}")

# Create datasets
train_dataset = HazmatDataset(
    data_dir='data/data_faster_rcnn/train_aug',
    annotations_file='data/data_faster_rcnn/train/annotations/instances_train.json',
    transforms=get_augmented_transform(train=True)
)

val_dataset = HazmatDataset(
    data_dir='data/data_faster_rcnn/val',
    annotations_file='data/data_faster_rcnn/val/annotations/instances_val.json',
    transforms=get_augmented_transform(train=False)
)

# Set the percentage of the training dataset to use (e.g. 0.x to 1)
train_percentage = 0.01

# Create a subset of the training dataset
train_dataset_subset = create_subset(train_dataset, train_percentage)

# Set the percentage of the val dataset to use (e.g. 0.x to 1)
val_percentage = 1

# Create a subset of the training dataset
val_dataset_subset = create_subset(val_dataset, val_percentage)

# amount of cpu cores
workers = 2

# Create data loaders
train_loader = DataLoader(
    train_dataset_subset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset_subset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

# Initialize model
num_classes = 2  # hazmat code and background

# Create ResNet-101 backbone with FPN
backbone = resnet_fpn_backbone('resnet101', pretrained=True)

# Define anchor generator for FPN
anchor_generator = AnchorGenerator(
    sizes=((32,), (64,), (128,), (256,), (512,)),
    aspect_ratios=((0.5, 1.0, 2.0),) * 5
)

# Multi-scale RoI pooling for FPN
roi_pooler = MultiScaleRoIAlign(
    featmap_names=['0', '1', '2', '3', '4'],
    output_size=7,
    sampling_ratio=2
)

print("initializing model...")
# Initialize Faster R-CNN with ResNet-101-FPN
model = FasterRCNN(
    backbone=backbone,
    num_classes=num_classes,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)

device = torch.device('cuda:1')
print(f"Training model on {device}")

# Move model to device
model.to(device)

# visualize_augmentations(train_dataset, num_samples=5)

# save the model as file so I can train it later and then destroy kernel session

In [ ]:
!nvidia-smi

In [ ]:
#!kill -9 48787
#torch.cuda.empty_cache()

# Training

In [ ]:
device_data = torch.device('cuda:1')
print(f"Training model on {device}")
scaler = GradScaler()
# Initialize optimizer and scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training loop
num_epochs = 1
train_metrics_map = []
best_val_map = float('-inf')

print("Starting training...")

# Create directory to store models and logs
directory_finetuned_model = create_directory()


for epoch in range(num_epochs):
    best_val_map = train_model(
        directory=directory_finetuned_model, 
        model=model, optimizer=optimizer, train_loader=train_loader, device=device, 
        train_metrics_list=train_metrics_map, best_val_map=best_val_map, lr_scheduler=lr_scheduler, 
        val_loader=val_loader, coco_val=coco_val, scaler=scaler, epoch=epoch
    )

